In [1]:
import numpy as np
import pandas as pd
import os
import re
import csv
import json
import matplotlib
import matplotlib.pyplot as plt
import plotly.express as px
import seaborn as sns
from collections import Counter

In [2]:
# Cargar el archivo de texto en un DataFrame
df = pd.read_csv("booksummaries.txt", header=None, sep="\t", names=['Wiki_ID', 'Freebase_ID', 'Title', 'Author', 'Pub_date', 'Genres', 'Summary'])

# Mostrar las primeras 5 filas del DataFrame
df.head(5)

,Wiki_ID,Freebase_ID,Title,Author,Pub_date,Genres,Summary
0,620,/m/0hhy,Animal Farm,George Orwell,1945-08-17,"{""/m/016lj8"": ""Roman \u00e0 clef"", ""/m/06nbt"":...","Old Major, the old boar on the Manor Farm, ca..."
1,843,/m/0k36,A Clockwork Orange,Anthony Burgess,1962,"{""/m/06n90"": ""Science Fiction"", ""/m/0l67h"": ""N...","Alex, a teenager living in near-future Englan..."
2,986,/m/0ldx,The Plague,Albert Camus,1947,"{""/m/02m4t"": ""Existentialism"", ""/m/02xlf"": ""Fi...",The text of The Plague is divided into five p...
3,1756,/m/0sww,An Enquiry Concerning Human Understanding,David Hume,NaN,NaN,The argument of the Enquiry proceeds by a ser...
4,2080,/m/0wkt,A Fire Upon the Deep,Vernor Vinge,NaN,"{""/m/03lrw"": ""Hard science fiction"", ""/m/06n90...",The novel posits that space around the Milky ...


In [3]:
# Mostrar información sobre el DataFrame
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 16559 entries, 0 to 16558
Data columns (total 7 columns):
 #   Column       Non-Null Count  Dtype 
---  ------       --------------  ----- 
 0   Wiki_ID      16559 non-null  int64 
 1   Freebase_ID  16559 non-null  object
 2   Title        16559 non-null  object
 3   Author       14177 non-null  object
 4   Pub_date     10949 non-null  object
 5   Genres       12841 non-null  object
 6   Summary      16559 non-null  object
dtypes: int64(1), object(6)
memory usage: 905.7+ KB


In [4]:
# Crear un DataFrame con el número de valores faltantes en cada columna
missing_values_df = pd.DataFrame(df.isna().sum(), columns=['recuento_faltantes'])

# Crear un gráfico de barras del recuento de valores faltantes utilizando Plotly
fig = px.bar(missing_values_df, x=missing_values_df.index, y='recuento_faltantes', 
             title='Valores faltantes en la base de datos')

# Personalizar el diseño del gráfico
fig.update_layout(xaxis_title='Columnas', yaxis_title='Recuento de valores faltantes', title_x=0.5)

# Mostrar el gráfico
fig.show()

In [5]:
# Convertir el género del formato json al formato de lista
def parse_genres(genre_str):
    try:
        return list(json.loads(genre_str).values())
    except (TypeError, json.JSONDecodeError):
        return []

# Aplicar la función parse_genres a la columna 'Genres'
df['Genres'] = df['Genres'].map(parse_genres)

# Mostrar las primeras 5 filas del DataFrame
df.head(5)

,Wiki_ID,Freebase_ID,Title,Author,Pub_date,Genres,Summary
0,620,/m/0hhy,Animal Farm,George Orwell,1945-08-17,"[Roman à clef, Satire, Children's literature, ...","Old Major, the old boar on the Manor Farm, ca..."
1,843,/m/0k36,A Clockwork Orange,Anthony Burgess,1962,"[Science Fiction, Novella, Speculative fiction...","Alex, a teenager living in near-future Englan..."
2,986,/m/0ldx,The Plague,Albert Camus,1947,"[Existentialism, Fiction, Absurdist fiction, N...",The text of The Plague is divided into five p...
3,1756,/m/0sww,An Enquiry Concerning Human Understanding,David Hume,NaN,[],The argument of the Enquiry proceeds by a ser...
4,2080,/m/0wkt,A Fire Upon the Deep,Vernor Vinge,NaN,"[Hard science fiction, Science Fiction, Specul...",The novel posits that space around the Milky ...


In [6]:
# Encontrar los autores más comunes
author_counts = df['Author'].value_counts().reset_index()
author_counts.columns = ['Autor', 'Recuento']

# Seleccionar los 10 principales autores con la cuenta más alta
top_authors = author_counts.nlargest(columns=["Recuento"], n=10)

# Restablecer el índice del DataFrame top_authors
top_authors.reset_index(drop=True, inplace=True)

# Ordenar los autores por Recuento
top_authors = top_authors.sort_values(by='Recuento', ascending=True)

# Crear un gráfico de barras de los recuentos de autor utilizando Plotly
fig = px.bar(top_authors, x='Recuento', y='Autor', 
             height=600, width=800, title='Autores más comunes')

# Personalizar el diseño del gráfico
fig.update_layout(xaxis_title='Recuento', yaxis_title='Autor', 
                  title_x=0.5, title_font_size=24)

# Mostrar el gráfico
fig.show()

# Calcular el porcentaje de libros con un autor asociado
total_books = len(df)
books_with_author = df['Author'].notna().sum()
percentage_books_with_author = (books_with_author / total_books) * 100

# # Imprimir los resultados
# print("Autores más comunes:")
# print(author_counts.head(10))
print(f"\nPorcentaje de libros con un autor asociado: {percentage_books_with_author:.2f}%")


Porcentaje de libros con un autor asociado: 85.62%


In [7]:
# Convertir la columna 'Pub_date' al formato datetime
df['Pub_date'] = pd.to_datetime(df['Pub_date'], errors='coerce')

# Encontrar el periodo de tiempo cubierto por los libros analizados
earliest_date = df['Pub_date'].min()
latest_date = df['Pub_date'].max()

# Encontrar los libros más recientes y antiguos
most_recent_book = df.loc[df['Pub_date'].idxmax()]
oldest_book = df.loc[df['Pub_date'].idxmin()]

# Imprimir los resultados
print(f"Periodo de tiempo cubierto: {earliest_date} a {latest_date}")
print(f"Libro más reciente: {most_recent_book['Title']} (publicado el: {most_recent_book['Pub_date']})")
print(f"Libro más antiguo: {oldest_book['Title']} (publicado el: {oldest_book['Pub_date']})")

Periodo de tiempo cubierto: 1678-02-01 00:00:00 a 2013-01-01 00:00:00
Libro más reciente: Dr. Sleep (publicado el: 2013-01-01 00:00:00)
Libro más antiguo: The Pilgrim's Progress (publicado el: 1678-02-01 00:00:00)


In [8]:
# Encontrar el índice de los títulos más largos y más cortos
longest_title_index = df['Title'].apply(len).idxmax()
shortest_title_index = df['Title'].apply(len).idxmin()

# Obtener los títulos más largos y más cortos
longest_title = df.loc[longest_title_index, 'Title']
shortest_title = df.loc[shortest_title_index, 'Title']

# Imprimir los resultados
print(f"Título más largo: {longest_title} (longitud: {len(longest_title)})")
print(f"Título más corto: {shortest_title} (longitud: {len(shortest_title)})")

Título más largo: Passionate Minds: The Great Love Affair of the Enlightenment, Featuring the Scientist Emilie du Chatelet, the Poet Voltaire, Sword Fights, Book Burnings, Assorted Kings, Seditious Verse, and the Birth of the Modern World (longitud: 220)
Título más corto: Q (longitud: 1)


In [9]:
# Contar los géneros individuales
genre_counts = Counter()
for genre_list in df['Genres']:
    genre_counts.update(genre_list)

# Crear un DataFrame con las columnas 'Género' y 'Recuento'
genre_count_df = pd.DataFrame({"Género": list(genre_counts.keys()), "Recuento": list(genre_counts.values())})

# Seleccionar los 10 principales géneros con la cuenta más alta
top_genres = genre_count_df.nlargest(columns=["Recuento"], n=10)

# Restablecer el índice del DataFrame top_genres
top_genres.reset_index(drop=True, inplace=True)

# Ordenar los géneros por Recuento
top_genres = top_genres.sort_values(by='Recuento', ascending=True)

# Crear un gráfico de barras de los recuentos de género utilizando Plotly
fig = px.bar(top_genres, x='Recuento', y='Género', 
             title='Géneros más comunes', height=600, width=800)

# Personalizar el diseño del gráfico
fig.update_layout(xaxis_title='Recuento', yaxis_title='Género', 
                  title_x=0.5, title_font_size=24, 
                  uniformtext_minsize=12, uniformtext_mode='hide')

# Mostrar el gráfico
fig.show()

# # Imprimir los recuentos de los 10 géneros principales utilizando el formato dado
# for index, row in top_genres.iterrows():
#     print(f"Género: {row['Genre']} (recuento: {row['Counts']})")

In [10]:
#Delete all the rows where summary and genre are NaN.
#df = df.dropna(subset=['Genres', 'Summary'])

## Pruebas

In [11]:
# Función para eliminar puntuación y convertir a minúsculas
def clean_summary(text):
    text = re.sub(r'[^\w\s]', '', text)  # Eliminar puntuación
    text = text.lower()  # Convertir a minúsculas
    return text

# Análisis de la sección 'Summary'
total_word_count = 0
word_counter = Counter()

for summary in df['Summary']:
    cleaned_summary = clean_summary(summary)
    words = cleaned_summary.split()
    
    total_word_count += len(words)
    word_counter.update(words)

# Calcular el recuento de palabras promedio por libro
average_word_count = total_word_count / len(df)

# Imprimir los resultados
print(f"Total de palabras en la sección 'Summary': {total_word_count}")
print(f"Palabras promedio por libro en la sección 'Summary': {average_word_count:.2f}")

# Analizar palabras individuales
single_word_count = sum(1 for count in word_counter.values() if count == 1)
print(f"Número de palabras individuales: {single_word_count}")

Total de palabras en la sección 'Summary': 7093752
Palabras promedio por libro en la sección 'Summary': 428.39
Número de palabras individuales: 67077


In [12]:
# !pip install spacy
#!python -m spacy download en_core_web_sm

import spacy

In [13]:
# Cargar el modelo de lenguaje inglés
nlp = spacy.load("en_core_web_sm")

# Obtener la lista de stopwords
stopwords = nlp.Defaults.stop_words

# Filtrar las palabras según la lista de stopwords
filtered_words = [word for word in word_counter if word not in stopwords]

# Crear un nuevo contador con las palabras filtradas
filtered_word_counter = Counter({word: word_counter[word] for word in filtered_words})

# Imprimir las palabras filtradas más comunes
most_common_filtered_words = filtered_word_counter.most_common(20)
print("Palabras más comunes no incluidas en la lista de stopwords:")
for word, count in most_common_filtered_words:
    print(f"{word}: {count}")

Palabras más comunes no incluidas en la lista de stopwords:
time: 11498
new: 10408
life: 10015
book: 8638
story: 8493
find: 8219
man: 8177
father: 8110
family: 7573
tells: 7318
home: 7241
later: 7204
finds: 7046
novel: 6902
world: 6897
mother: 6783
house: 6711
years: 6672
death: 6466
takes: 6229


In [14]:
# !pip install nltk

import nltk
from nltk.stem import PorterStemmer

In [15]:
# Cargar el modelo de lenguaje inglés
nlp = spacy.load("en_core_web_sm")

# Inicializar el stemmer
ps = PorterStemmer()

# Limpiar y tokenizar los resúmenes
cleaned_summaries = [clean_summary(summary) for summary in df['Summary']]
tokenized_summaries = [summary.split() for summary in cleaned_summaries]

# Stemming
stemmed_words = set()
for tokens in tokenized_summaries:
    stemmed_tokens = [ps.stem(token) for token in tokens if token not in stopwords]
    stemmed_words.update(stemmed_tokens)

# Lematización
lemmatized_words = set()
for tokens in tokenized_summaries:
    lemmatized_tokens = [token.lemma_ for token in nlp(" ".join(tokens)) if token.text not in stopwords]
    lemmatized_words.update(lemmatized_tokens)

# Calcular la reducción en el número de palabras únicas
initial_unique_words = len(word_counter)
stemmed_unique_words = len(stemmed_words)
lemmatized_unique_words = len(lemmatized_words)

stemming_reduction = initial_unique_words - stemmed_unique_words
lemmatization_reduction = initial_unique_words - lemmatized_unique_words

print(f"Palabras únicas iniciales: {initial_unique_words}")
print(f"Palabras únicas después del stemming: {stemmed_unique_words} (reducción: {stemming_reduction})")
print(f"Palabras únicas después de la lematización: {lemmatized_unique_words} (reducción: {lemmatization_reduction})")

Palabras únicas iniciales: 156574
Palabras únicas después del stemming: 113374 (reducción: 43200)
Palabras únicas después de la lematización: 135614 (reducción: 20960)
